# Chapter 12: Custom Models and Training with TensorFlow

Sebagian besar pengembangan model deep learning dapat diselesaikan menggunakan API tingkat tinggi seperti `tf.keras`. Namun, pada skenario lanjutan—terutama dalam penelitian dan eksperimen arsitektur baru—pendekatan tersebut sering kali kurang fleksibel.

Chapter ini membahas bagaimana TensorFlow memungkinkan kita untuk bekerja pada level yang lebih rendah, sehingga pengembang dapat membangun komponen neural network secara manual. Dengan pendekatan ini, kita memperoleh kontrol penuh terhadap:
- Cara model menghitung loss
- Bagaimana bobot diperbarui
- Struktur internal layer dan model

Pemahaman ini sangat penting bagi praktisi deep learning yang ingin mengimplementasikan metode baru yang belum tersedia dalam API standar.


## Ruang Lingkup Pembahasan Chapter 12

Topik-topik utama yang akan dibahas pada bab ini meliputi:

1. Penggunaan TensorFlow sebagai alternatif NumPy untuk komputasi numerik  
2. Pembuatan fungsi loss kustom sesuai kebutuhan model  
3. Implementasi custom layer dan custom model menggunakan subclassing  
4. Mekanisme automatic differentiation melalui GradientTape  
5. Penulisan training loop manual untuk kendali penuh proses pelatihan  
6. Optimasi performa dengan TensorFlow Functions dan computational graph

Seluruh konsep ini merupakan fondasi penting dalam pengembangan model deep learning tingkat lanjut.


## 1. Menggunakan TensorFlow seperti NumPy

TensorFlow menyediakan struktur data bernama *Tensor* yang secara konseptual mirip dengan array NumPy. Perbedaannya terletak pada kemampuan TensorFlow untuk:
- Menjalankan komputasi pada GPU atau TPU
- Mendukung komputasi terdistribusi
- Menghitung gradien secara otomatis

Tensor dapat bersifat immutable (`tf.constant`) maupun mutable (`tf.Variable`). Dalam konteks neural network, bobot model selalu direpresentasikan menggunakan `tf.Variable` karena nilainya berubah selama proses training.


In [ ]:
import tensorflow as tf
import numpy as np

# Membuat Tensor
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print("Tensor:\n", t)

# Operasi Dasar
print("Tambah 10:\n", t + 10)
print("Square:\n", tf.square(t))

# Variabel (Mutable - digunakan untuk bobot model)
v = tf.Variable([[1., 2.], [3., 4.]])
v.assign(2 * v)
v[0, 1].assign(42)
print("Variable:\n", v)



## 2. Custom Loss Functions

Fungsi loss berperan sebagai indikator utama performa model selama training. Dalam beberapa kasus, fungsi loss bawaan Keras tidak sepenuhnya sesuai dengan kebutuhan permasalahan.

Dengan mendefinisikan fungsi loss sendiri, kita dapat:
- Mengontrol sensitivitas terhadap outlier
- Menggabungkan beberapa komponen penalti
- Menyesuaikan perilaku loss terhadap domain tertentu

Sebagai contoh, Huber Loss sering digunakan karena lebih stabil dibandingkan Mean Squared Error ketika data mengandung outlier.


In [ ]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# Penggunaan dalam Model
# model.compile(loss=huber_fn, optimizer="nadam")

## 3. Custom Layers dan Custom Models

Keras memungkinkan pengembang untuk membuat layer kustom dengan melakukan subclassing terhadap `keras.layers.Layer`. Pendekatan ini memberikan fleksibilitas tinggi, terutama ketika:
- Operasi yang dibutuhkan tidak tersedia dalam layer standar
- Struktur perhitungan dalam layer bersifat tidak konvensional
- Model perlu dioptimalkan secara spesifik

Dengan mendefinisikan metode `build()` dan `call()`, kita dapat mengontrol pembuatan bobot serta alur komputasi forward pass secara eksplisit.


In [ ]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        # Membuat bobot (weights)
        self.kernel = self.add_weight(name="kernel",
                                      shape=[batch_input_shape[-1], self.units],
                                      initializer="glorot_normal")
        self.bias = self.add_weight(name="bias", shape=[self.units],
                                    initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

# Menggunakan layer kustom dalam model
model = tf.keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=[8]),
    MyDense(1)
])



## 4. Autodifferentiation dengan GradientTape

Salah satu kekuatan utama TensorFlow adalah kemampuannya dalam menghitung turunan secara otomatis melalui mekanisme *automatic differentiation*.

Objek `tf.GradientTape` mencatat seluruh operasi matematika yang terjadi selama eksekusi forward pass. Informasi ini kemudian digunakan untuk menghitung gradien terhadap variabel tertentu saat proses backpropagation.

Pendekatan ini memungkinkan perhitungan gradien yang fleksibel, bahkan untuk fungsi matematika yang kompleks dan tidak terstruktur.


In [ ]:
def f(w1, w2):
    return 3 * w1**2 + 2 * w1 * w2

w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print("Gradients [dz/dw1, dz/dw2]:", gradients)

## 5. Custom Training Loops

Meskipun metode `.fit()` sangat praktis, terdapat situasi di mana kita memerlukan kendali penuh terhadap setiap langkah training. Dalam kondisi tersebut, custom training loop menjadi solusi utama.

Dengan menulis training loop secara manual, kita dapat:
- Mengatur update bobot secara eksplisit
- Menggabungkan beberapa objective function
- Mengimplementasikan algoritma training non-standar

Pendekatan ini umum digunakan pada reinforcement learning, meta-learning, dan penelitian eksperimental.


In [ ]:
# Skenario sederhana Custom Training Loop
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error

def train_step(model, X_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred = model(X_batch)
        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

## 6. TensorFlow Functions dan Computational Graph

Secara default, TensorFlow berjalan dalam *eager execution mode*, yang memudahkan debugging namun kurang optimal dari sisi performa.

Decorator `@tf.function` memungkinkan konversi fungsi Python menjadi computational graph TensorFlow. Dengan pendekatan ini:
- Overhead Python dapat diminimalkan
- Eksekusi menjadi lebih cepat
- Kode lebih siap untuk deployment produksi

Namun, penggunaan `@tf.function` memerlukan perhatian ekstra karena tidak semua operasi Python dapat dikonversi menjadi graph.


In [ ]:
@tf.function
def tf_cube(x):
    return x ** 3

print("Cube of 2:", tf_cube(tf.constant(2.0)))

## Kesimpulan Chapter 12

Chapter ini menekankan pentingnya memahami TensorFlow pada level yang lebih mendalam. Dengan menguasai custom components, autodifferentiation, dan training loop manual, pengembang tidak lagi terbatas pada API siap pakai.

Kemampuan ini sangat penting bagi praktisi dan peneliti yang ingin:
- Mengimplementasikan arsitektur neural network baru
- Mengadaptasi algoritma dari paper penelitian
- Mengoptimalkan performa model untuk kasus khusus

Dengan fondasi ini, kita siap melangkah ke pengembangan deep learning tingkat lanjut.
